In [2]:
from qaravan.core import *
from qaravan.tensorQ import *

In [9]:
# generate a random order of canonizing and check center for each choice of center 

n = 5 
center_opts = np.arange(n)
np.random.shuffle(center_opts)

for center in center_opts:
    print(center)
    mps.canonize(center)
    check_center(mps, center, verbose=True)

2
Site 0: L
Site 1: L
Site 2: N
Site 3: R
Site 4: R
3
Site 0: L
Site 1: L
Site 2: L
Site 3: N
Site 4: R
4
Site 0: L
Site 1: L
Site 2: L
Site 3: L
Site 4: L
1
Site 0: L
Site 1: N
Site 2: R
Site 3: R
Site 4: R
0
Site 0: R
Site 1: R
Site 2: R
Site 3: R
Site 4: R


In [10]:
# test environments
n = 5
mps = random_mps(n)

re = mps.compute_right_envs()
le = mps.compute_left_envs()
for i in range(n-1): 
    assert np.allclose(ncon((le[i+1], re[i]), ([1,2],[1,2])), mps.norm())

In [11]:
# test environment with two MPS 
n = 5
mps1 = random_mps(n)
mps2 = random_mps(n)

true_overlap = mps2.to_vector().conj().T @ mps1.to_vector()

re = mps1.compute_right_envs(mps2)
le = mps1.compute_left_envs(mps2)
for i in range(n-1):
    assert np.allclose(ncon((le[i+1], re[i]), ([1,2],[1,2])), true_overlap)

In [12]:
# test overlap method 
n = 6
mps1 = random_mps(n)
mps2 = random_mps(n)
mps_overlap = mps1.overlap(mps2, scaled=False)

sv1 = mps1.to_vector() 
sv2 = mps2.to_vector()
direct_overlap = sv2.conj().T @ sv1 

assert np.allclose(mps_overlap, direct_overlap)

In [17]:
# test fast_measure()

n = 4
meas_sites = [1,3]
mps = random_mps(n)

sv = mps.to_vector()
rdm = rdm_from_sv(sv, meas_sites)
exact_density_vec = np.diag(rdm).real


num_sample_opts = [100, 1000, 10000]
for num_samples in num_sample_opts:
    mps_shots = [mps.fast_measure(meas_sites) for _ in range(num_samples)]
    mps_density = shots_to_density(mps_shots)
    mps_density_vec = shots_to_density_vec(mps_shots)
    assert np.linalg.norm(exact_density_vec - mps_density_vec, ord=2) < l2_threshold(num_samples, 2**len(meas_sites), delta=1e-3)

In [ ]:
# test one_rdm()
n=5
mps = random_mps(n)

sv = mps.to_vector()
true_rdm = rdm_from_sv(sv, [2])
mps_rdm = mps.one_rdm(2)
assert np.allclose(true_rdm, mps_rdm)

In [ ]:
# test one_local_expectation()
n=5
mps = random_mps(n)
sv = mps.to_vector()
op = random_hermitian_op(1)
true_exp = np.trace(op @ rdm_from_sv(sv, [2])).real
mps_exp = mps.one_local_expectation(2, op)
assert np.allclose(true_exp, mps_exp)